## 1. Download and label images with Label Studio

**1.1 Install Required Packages**
```bash
# Core packages
pip install label-studio
pip install label-studio-converter
pip install azure-cognitiveservices-vision-customvision
pip install azure-storage-blob
pip install coco2customvision
pip install pylabel
pip install requests
pip install python-dotenv
```


**1.2 Start Label Studio**
```bash
# Set environment variables

# Windows
set LOCAL_FILES_SERVING_ENABLED=true
set LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT="C:\path\to\exported_data"     # If using images that have already been labeled

# Git Bash/ Unix-like / Linux
export LOCAL_FILES_SERVING_ENABLED=true
export LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT="C:\path\to\exported_data"     # If using images that have already been labeled

# Start Label Studio
label-studio
```
**NOTE:** Make sure you have LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT set to the correct PATH each time your start label-studio!!!

**1.3 Set Up Project in Label Studio**
1. Open http://localhost:8080
2. Create a new project
3. Go to Settings → Cloud Storage
4. Click Add Source Storage → Local files
5. Set Absolute local path to: /path/to/exported_data/images
6. Click Add storage
7. Then set Target Cloud Storage for annotation output
8. Finally go to the `Labeling Interface` tab and add all of your bounding box labels
    - Manual Example:
```xml
<View>
  <Image name="image" value="$image"/>
  <RectangleLabels name="label" toName="image">
    <Label value="YourClass1" background="red"/>
    <Label value="YourClass2" background="blue"/>
    <!-- Add your actual class names here -->
  </RectangleLabels>
</View>
```

**1.4 Import Data**
1. In your project, go to Data Manager
2. Click Import
3. Select the .json file with labels (If using images that have already been labeled)
4. Your images and annotations should now be loaded


**1.5 Export Annotations**
1. In Label Studio, go to your project
2. Click Export
3. Select COCO format
4. Download the exported file

## 2. Send Files to Azure Custom Vision

In [ ]:
import os
import json
from decimal import Decimal
from pathlib import Path, PurePath
from dotenv import load_dotenv
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region
from msrest.authentication import ApiKeyCredentials

load_dotenv()

# --- CONFIGURATION ---
ENDPOINT = os.getenv("ENDPOINT")
TRAINING_KEY = os.getenv("TRAINING_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")
COCO_ANNOTATION_FILE = r""
IMAGES_DIR = r""
MAPPING_FILE = r""

# --- AUTHENTICATION ---
credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# --- LOAD COCO ANNOTATIONS ---
with open(COCO_ANNOTATION_FILE, 'r') as f:
    coco = json.load(f)

# Build image and annotation lookup
images = {img['id']: img for img in coco['images']}
annotations_by_image = {}
for ann in coco['annotations']:
    annotations_by_image.setdefault(ann['image_id'], []).append(ann)

# Map COCO categories to Custom Vision tags
category_name_to_id = {cat['name']: cat['id'] for cat in coco['categories']}
# Get Custom Vision tags (assumes tags already created in project)
cv_tags = {tag.name: tag for tag in trainer.get_tags(PROJECT_ID)}

# --- MAPPING FUNCTION ---
def save_mapping(image_path, cv_image_id, regions_data, mapping_file):
    mapping = {}
    if os.path.exists(mapping_file):
        with open(mapping_file, 'r') as f:
            mapping = json.load(f)
    mapping[image_path] = {
        "custom_vision_id": cv_image_id,
        "filename": os.path.basename(image_path),
        "full_path": os.path.abspath(image_path),
        "region_count": len(regions_data),
        "upload_status": "success"
    }
    with open(mapping_file, 'w') as f:
        json.dump(mapping, f, indent=2)

# --- MAIN UPLOAD LOOP ---
for image_id, image_info in images.items():
    img_filename = os.path.basename(image_info['file_name'])
    img_path = os.path.join(IMAGES_DIR, img_filename)
    assert os.path.exists(img_path), f"File does not exist: {img_path}"

    width, height = image_info['width'], image_info['height']
    regions = []
    anns = annotations_by_image.get(image_id, [])
    for ann in anns:
        cat_id = ann['category_id']
        cat_name = next((c['name'] for c in coco['categories'] if c['id'] == cat_id), None)
        if cat_name not in cv_tags:
            print(f"Warning: Category '{cat_name}' not found in Custom Vision tags. Skipping annotation.")
            continue
        tag_id = cv_tags[cat_name].id
        # COCO bbox: [x_min, y_min, width, height]
        x, y, w, h = ann['bbox']
        x_norm = min(Decimal(x / width), 1)
        y_norm = min(Decimal(y / height), 1)
        w_norm = min(Decimal(w / width), 1 - x_norm)
        h_norm = min(Decimal(h / height), 1 - y_norm)
        regions.append(Region(
            tag_id=tag_id,
            left=float(x_norm),
            top=float(y_norm),
            width=float(w_norm),
            height=float(h_norm)
        ))

    with open(img_path, "rb") as image_contents:
        image_and_annotations = [
            ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)
        ]

    upload_result = trainer.create_images_from_files(
        PROJECT_ID,
        ImageFileCreateBatch(images=image_and_annotations)
    )

    if not upload_result.is_batch_successful:
        print(f"Image upload failed for {img_path}.")
        for image in upload_result.images:
            print("Image status:", image.status)
            print("Regions:", regions)
    else:
        cv_image_id = upload_result.images[0].image.id
        save_mapping(img_path, cv_image_id, regions, MAPPING_FILE)

print("Upload complete.")

## 3. Train and evaluate your model in Azure Custom Vision

## 4. Add new images to local training data if necessary

## 5. Use your current Azure Model to predict the bounding boxes on your new trianing images

In [ ]:
from pathlib import Path
import requests, json, uuid
from PIL import Image

# ROOT is the path to your new training images
ROOT = Path(r"")
HEADERS = {
    "Prediction-Key": os.getenv("PREDICTION_KEY"),
    "Content-Type": "application/octet-stream"
}
ENDPOINT = os.getenv("ENDPOINT")
PROJECT_ID = os.getenv("PROJECT_ID")
ITERATION_NAME = os.getenv("ITERATION_NAME")
CONFIDENCE_THRESHOLD = 0.82

EXPORT_DIR = Path("azure_cv_annotations.json")
EXPORT_DIR.mkdir(exist_ok=True)

EXPORT_DIR = Path("azure_cv_annotations.json")
EXPORT_DIR.mkdir(exist_ok=True)

for img_path in ROOT.glob("*.png"):
    # Load image and get dimensions
    with Image.open(img_path) as img:
        width, height = img.size

    # Send image to Custom Vision API
    data = img_path.read_bytes()
    resp = requests.post(
        f"{ENDPOINT}/customvision/v3.0/Prediction/{PROJECT_ID}/detect/iterations/{ITERATION_NAME}/image",
        headers=HEADERS,
        data=data
    )
    preds = resp.json()["predictions"]

    img_path_uri = "file:" + img_path.as_posix().replace(" ", "%20")

    # Build JSON asset
    json_asset = {
        "asset": {
            "format": "png",
            "id": str(uuid.uuid4()),
            "name": img_path.name,
            "path": img_path_uri,
            "size": {"width": width, "height": height},
            "state": 2,      # tagged
            "type": 1        # image
        },
        "regions": [{
            "id": str(uuid.uuid4()),
            "tags": [p["tagName"]],
            "boundingBox": {
                "left": p["boundingBox"]["left"],
                "top": p["boundingBox"]["top"],
                "width": p["boundingBox"]["width"],
                "height": p["boundingBox"]["height"]
            }
        } for p in preds if p["probability"] > CONFIDENCE_THRESHOLD],
    }

    # Save JSON to file
    output_file = EXPORT_DIR / f"{img_path.stem}-asset.json"
    output_file.write_text(json.dumps(json_asset, indent=2))

**Convert the Azure label files to a format Label Studio can read**

In [ ]:
import json
import os
import glob
from pathlib import Path
from urllib.parse import unquote
from PIL import Image

def get_image_size(path: Path) -> tuple[int, int]:
    with Image.open(path) as img:
        return img.width, img.height


def convert_json_directory_to_labelstudio(
    json_dir: str,
    output_file: str,
    image_root_url="/data/local-files/?d=images",
    copy_images_to: str | None = None
):
    json_files = glob.glob(os.path.join(json_dir, "*.json"))
    print(f"Found {len(json_files)} JSON files")

    if not json_files:
        return

    label_studio_tasks, all_tags = [], set()
    processed_count = 0

    for jf in json_files:
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)

        asset, regions = data.get("asset", {}), data.get("regions", [])
        if not regions:
            continue

        image_name = asset.get("name", "")
        image_path = Path(unquote(asset.get("path", "").replace("file:", "")))

        if not image_name or not image_path.exists():
            continue

        # Copy image if requested
        if copy_images_to:
            dest = Path(copy_images_to) / image_name
            dest.parent.mkdir(parents=True, exist_ok=True)
            if not dest.exists():
                import shutil
                shutil.copy2(image_path, dest)
        else:
            dest = image_path  # original location

        # Determine dimensions once per image
        orig_w, orig_h = get_image_size(dest)

        # Build task
        task = {
            "data": {"image": f"{image_root_url}/{image_name}"},
            "annotations": [{"result": []}]
        }

        for region in regions:
            bbox, tags = region.get("boundingBox", {}), region.get("tags", [])
            if not bbox or not tags:
                continue

            x_pct, y_pct = bbox.get("left", 0) * 100, bbox.get("top", 0) * 100
            w_pct, h_pct = bbox.get("width", 0) * 100, bbox.get("height", 0) * 100

            all_tags.update(tags)
            for tag in tags:
                task["annotations"][0]["result"].append({
                    "id": region.get("id") or f"region_{len(task['annotations'][0]['result'])}",
                    "value": {
                        "x": x_pct,
                        "y": y_pct,
                        "width": w_pct,
                        "height": h_pct,
                        "rectanglelabels": [tag],
                        "original_width": orig_w,
                        "original_height": orig_h,
                        "rotation": 0
                    },
                    "from_name": "label",
                    "to_name": "image",
                    "type": "rectanglelabels",
                    "origin": "manual"
                })

        label_studio_tasks.append(task)
        processed_count += 1

    os.makedirs(Path(output_file).parent, exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(label_studio_tasks, f, indent=2, ensure_ascii=False)

    print(f"Converted {processed_count} files ➜ {output_file}")
    print(f"Labels found: {sorted(all_tags)}")

    # Generate XML config
    cfg_path = Path(output_file).with_suffix("_config.xml")
    xml = "<View>\n  <Image name=\"image\" value=\"$image\"/>\n  <RectangleLabels name=\"label\" toName=\"image\">\n"
    xml += "\n".join([f'    <Label value="{t}" background="#{hash(t)%0xFFFFFF:06x}"/>' for t in sorted(all_tags)])
    xml += "\n  </RectangleLabels>\n</View>"
    cfg_path.write_text(xml, encoding="utf-8")
    print(f"Wrote interface config ➜ {cfg_path}")

    return label_studio_tasks, sorted(all_tags)

def batch_convert_with_image_copy(json_dir, output_dir="label_studio_data", images_subdir="images"):
    os.makedirs(output_dir, exist_ok=True)
    images_dir = os.path.join(output_dir, images_subdir)
    
    tasks, tags = convert_json_directory_to_labelstudio(
        json_dir=json_dir,
        output_file=os.path.join(output_dir, "label_studio_tasks.json"),
        image_root_url=f"/data/local-files/?d={images_subdir}",  # CORRECT: /data/local-files/?d=images
        copy_images_to=images_dir
    )
    
    print(f"\n=== SETUP INSTRUCTIONS ===")
    print(f"1. Set environment variables:")
    print(f"   export LOCAL_FILES_SERVING_ENABLED=true")
    print(f"   export LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=\"{os.path.abspath(output_dir)}\"")
    print(f"2. Start Label Studio")
    print(f"3. Create project with XML config from: {os.path.join(output_dir, 'label_studio_tasks_config.xml')}")
    print(f"4. Set local storage path to: {os.path.abspath(images_dir)}")
    print(f"5. Import: {os.path.join(output_dir, 'label_studio_tasks.json')}")
    
    return tasks, tags

if __name__ == "__main__":
    json_directory = r""
    
    batch_convert_with_image_copy(
        json_dir=json_directory,
        output_dir="",
        images_subdir="images"
    )

## *Back to step one*

#### Extra Tools

#### 1. Loading all images from Azure Custom Vision with labels to be checked in Label Studio

In [ ]:
import os
from dotenv import load_dotenv
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import json
import requests
from urllib.parse import urlparse
import time

load_dotenv()

ENDPOINT = os.getenv("ENDPOINT")
TRAINING_KEY = os.getenv("TRAINING_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")
ITERATION_ID = None # or None for latest
IMAGES_PER_PAGE = 256

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

def export_annotations_only():
    # Get all images with pagination
    all_images = []
    skip = 0
    take = 256
    
    while True:
        print(f"Fetching images {skip} to {skip + take}...")
        images_batch = trainer.get_images(PROJECT_ID, skip=skip, take=take)
        
        if hasattr(images_batch, 'images'):
            batch_list = images_batch.images  
        else:
            batch_list = images_batch
        
        if not batch_list:
            break
            
        all_images.extend(batch_list)
        skip += take
        
        if len(batch_list) < take:
            break
    
    print(f"Found {len(all_images)} total images")
    
    # Create export data
    export_data = {
        "project_info": {
            "total_images": len(all_images),
            "images_with_annotations": 0
        },
        "categories": [],
        "images": []
    }
    
    # Get project tags
    tags = trainer.get_tags(PROJECT_ID)
    for tag in tags:
        export_data["categories"].append({
            "id": tag.id,
            "name": tag.name
        })
    
    # Process images
    for img_idx, image in enumerate(all_images):
        image_data = {
            "id": image.id,
            "index": img_idx,
            "width": image.width,
            "height": image.height,
            "original_uri": image.original_image_uri,
            "regions": []
        }
        
        if hasattr(image, 'regions') and image.regions:
            export_data["project_info"]["images_with_annotations"] += 1
            
            for region in image.regions:
                region_data = {
                    "tag_name": region.tag_name if hasattr(region, 'tag_name') else str(region.tag_id),
                    "left": region.left,
                    "top": region.top,
                    "width": region.width,
                    "height": region.height
                }
                image_data["regions"].append(region_data)
        
        export_data["images"].append(image_data)
    
    # Save annotation data
    os.makedirs('exported_data', exist_ok=True)
    with open('exported_data/custom_vision_export.json', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"Exported annotation data for {len(all_images)} images")
    print(f"Images with annotations: {export_data['project_info']['images_with_annotations']}")
    print(f"Categories: {len(export_data['categories'])}")

if __name__ == "__main__":
    export_annotations_only()

**Convert to Label Studio readable format with mapping to images**

In [ ]:
import json
import uuid

# Load Azure Custom Vision export
with open('custom_vision_export.json', 'r') as f:
    azure_data = json.load(f)

# Load mapping file
with open('mapping.json', 'r') as f:
    mapping = json.load(f)

# Build a reverse mapping: custom_vision_id -> mapping info
id_to_mapping = {v['custom_vision_id']: v for v in mapping.values()}

# Set your Label Studio image prefix (adjust as needed)
LABEL_STUDIO_IMAGE_PREFIX = '/data/local-files/?d=images/'

# Set your desired output image size (Label Studio expects the original size)
# If you have the true original size, use it. Otherwise, use the width/height from Azure.
def get_original_size(image):
    return image['width'], image['height']

label_studio_tasks = []
task_id = 1
annotation_id = 1

for image in azure_data['images']:
    custom_vision_id = image['id']
    mapping_info = id_to_mapping.get(custom_vision_id)
    if not mapping_info:
        print(f"Warning: No mapping found for image id {custom_vision_id}")
        continue

    filename = mapping_info['filename']
    # Compose the Label Studio image path
    ls_image_path = LABEL_STUDIO_IMAGE_PREFIX + filename

    width, height = get_original_size(image)

    # Build Label Studio annotation results
    results = []
    for region in image.get('regions', []):
        # Azure gives normalized coordinates (0-1), Label Studio expects percent (0-100)
        x = region['left'] * 100
        y = region['top'] * 100
        w = region['width'] * 100
        h = region['height'] * 100

        result = {
            "original_width": width,
            "original_height": height,
            "image_rotation": 0,
            "value": {
                "x": x,
                "y": y,
                "width": w,
                "height": h,
                "rotation": 0,
                "rectanglelabels": [region['tag_name']]
            },
            "id": str(uuid.uuid4()),
            "from_name": "label",
            "to_name": "image",
            "type": "rectanglelabels",
            "origin": "manual"
        }
        results.append(result)

    # Build the annotation object
    annotation = {
        "id": annotation_id,
        "completed_by": 1,
        "result": results,
        "was_cancelled": False,
        "ground_truth": True,
        "created_at": "",
        "updated_at": "",
        "draft_created_at": None,
        "lead_time": None,
        "prediction": {},
        "result_count": len(results),
        "unique_id": str(uuid.uuid4()),
        "import_id": None,
        "last_action": None,
        "bulk_created": False,
        "task": task_id,
        "project": 4,
        "updated_by": 1,
        "parent_prediction": None,
        "parent_annotation": None,
        "last_created_by": None
    }

    # Build the task object
    task = {
        "id": task_id,
        "annotations": [annotation],
        "file_upload": "",
        "drafts": [],
        "predictions": [],
        "data": {
            "image": ls_image_path
        },
        "meta": {},
        "created_at": "",
        "updated_at": "",
        "inner_id": task_id,
        "total_annotations": 1,
        "cancelled_annotations": 0,
        "total_predictions": 0,
        "comment_count": 0,
        "unresolved_comment_count": 0,
        "last_comment_updated_at": None,
        "project": 4,
        "updated_by": 1,
        "comment_authors": []
    }

    label_studio_tasks.append(task)
    task_id += 1
    annotation_id += 1

# Write to output file
with open('label_studio_tasks.json', 'w') as f:
    json.dump(label_studio_tasks, f, indent=2)

print(f"Exported {len(label_studio_tasks)} tasks to label_studio_tasks.json")